In [32]:
#pip install plotly==5.6.0

In [33]:
import plotly.express as px
import plotly.io as pio
pio.templates
import plotly.graph_objects as go
import numpy as np
from plotly.subplots import make_subplots
import pandas as pd
import datetime
from datetime import date, timedelta
import re

In [34]:
def cut(x):
    return x[:-3]

In [35]:
def format_date1(x):
    return datetime.datetime.strptime(x, "%Y-%m-%d %X").strftime("%Y-%m-%d")

In [44]:
# New Subscribers Over Time

#TODO Why not purchased_at inplace of subscriber_since?

# *** NEW SUBSCRIBERS OVER TIME *** 

# format new subscriptions data (using only subscriber_since data and grouping by date)
all_data_df = pd.read_csv("subscriptions.csv")
all_data_df = all_data_df.rename(columns={'subscriber_since': 'Date Purchased'})
new_subs_df = pd.DataFrame(all_data_df['Date Purchased'])
new_subs_df = new_subs_df.dropna(how = 'all')
new_subs_df["Date Purchased"] = new_subs_df["Date Purchased"].apply(cut)
new_subs_df["Date Purchased"] = new_subs_df["Date Purchased"].apply(format_date1)
num_new_subs = new_subs_df.groupby(['Date Purchased']).size().to_frame(name = 'Number of Subscribers').reset_index()
# Add a new column, 'Number of Subscribers From Trialers'. Initialize entire column to 0 
num_new_subs['Number of Subscribers (Trialers)'] = 0
print(num_new_subs.head(), "\n")

# **** NEW TRIALERS OVER TIME ***

# format new subscriptions from trialers data (using only subscriber since_data and was_trialer and grouping by date)
new_subs_from_trial_df = all_data_df[['Date Purchased', 'was_trial']].copy()
new_subs_from_trial_df = new_subs_from_trial_df.dropna()
print(new_subs_from_trial_df.head(), "\n\n", "Trials over time", "\n")
new_subs_from_trial_df = new_subs_from_trial_df[new_subs_from_trial_df.was_trial != "f"]
print(new_subs_from_trial_df.head(), "\n\n", "Frequency Table: Trialers Over Time", "\n")
# Delete the 'was_trial' column, currently holding trues-only, and build a "trues-only" frequency
# table. 
del new_subs_from_trial_df["was_trial"]
new_subs_from_trial_df["Date Purchased"] = new_subs_from_trial_df["Date Purchased"].apply(cut)
new_subs_from_trial_df["Date Purchased"] = new_subs_from_trial_df["Date Purchased"].apply(format_date1)
num_new_subs_from_trial = new_subs_from_trial_df.groupby(['Date Purchased']).size().to_frame(name = 'Number of Subscribers (Trialers)').reset_index()
print(num_new_subs_from_trial.head(), "\n", "Total Subscriptions and Trials Component Over Time", "\n")

# *** COMBINED: NEW SUBSCRIBERS AND TRIALERS OVER TIME ***

# combine two dfs
all_subs = pd.merge(num_new_subs, num_new_subs_from_trial, on="Date Purchased", how="outer")
# There'll two columns with the lede, "Number of Subscribers (Trialers)..."
del all_subs["Number of Subscribers (Trialers)_x"]
all_subs = all_subs.rename(columns={'Number of Subscribers (Trialers)_y': 'Number of Subscribers (Trialers)'})
all_subs['Number of Subscribers (Trialers)'].fillna(0, inplace=True)

#  plot data
df = all_subs
fig = make_subplots(rows=1, cols=1, subplot_titles=(""))
fig.add_trace(go.Scatter(x=df["Date Purchased"], y=df["Number of Subscribers"], name="Number of Subscribers"),
                         row=1, col=1)
fig.add_trace(go.Scatter(x=df["Date Purchased"], y=df["Number of Subscribers (Trialers)"], name="Number of Trialers"), 
                         row=1, col=1)

fig.update_xaxes(title_text="Date Purchased", row=1, col=1)
fig.update_yaxes(title_text="Number", row=1, col=1)
fig.update_layout(title_text="Subscriptions Over Time | Trial Subscriptions Overlayed")

fig.show()
#fig.write_html('index.html', auto_open=True)


  Date Purchased  Number of Subscribers  Number of Subscribers (Trialers)
0     2021-10-14                      3                                 0
1     2021-10-15                      5                                 0
2     2021-10-16                      1                                 0
3     2021-10-17                      2                                 0
4     2021-10-18                      1                                 0 

              Date Purchased was_trial
411   2021-10-21 10:35:42-05         f
1051  2021-11-07 20:56:15-06         f
1139  2021-10-29 18:04:35-05         f
1200  2021-11-01 16:28:14-06         f
1207  2021-10-22 15:33:33-05         f 

 Trials over time 

              Date Purchased was_trial
1242  2021-11-23 15:35:28-06         t
1243  2021-11-16 16:29:32-06         t
1691  2021-11-25 10:38:33-06         t
1747  2021-12-05 18:14:57-06         t
1749  2021-12-03 14:52:27-06         t 

 Frequency Table: Trialers Over Time 

  Date Purchased  Numbe